__Sphero Robotic Controller Experiments__

This project is an experiment in different automatic controllers for the sphero robot. 

The API it uses for communication is the bluecove API and a modified version of the 
[__nicklasgav Sphero-Desktop-API__](https://github.com/nicklasgav/Sphero-Desktop-API) 
which includes some additional changes for the sensor streaming api, the changes are located here in the [fork of Sphero-Desktop-API](https://github.com/cxd/Sphero-Desktop-API.git).

The following page contains some notes as to the implementation of the __RandomController__ and the __BinomialController__.

The __RandomController__ is very simple, the method _changeDirection_
is used to change direction using a _randomHeading_.

Given the __RollCommand__ takes a velocity between $0..1$ and a heading in degrees the new heading is
calculated as
$$
randDegrees = rand(0,1) \times 2 \pi \frac{180}{\pi}
$$
If the _changeDirection_ method is triggered due to a collision in the prior iteration it adds 90 degrees to the last heading.
$$
randDegrees = \theta  \frac{180}{\pi} + 90
$$


The idea behind the __BinomialController__ is to assign a likelihood of collision $P_\theta(y|\pi) \sim Binom(y, n, \pi)$ for each whole degree heading $\theta \in 1..360$ within a sliding window of $n=N$ (configured currently to 10) using the binomial  distribution $Binom_\theta(n,y,\pi)$ where $\pi$ is the expected proportion of the collision event. The beta-binomial distribution is used to update the initial uniform parameter prior $P_\theta(\pi)$ based on observation of either a collision, or limited _travel_ between the current state (which includes the position read from the odometer) and the last state. 

When either a collision occurs or when the distance travelled is less than a configured threshold, the update rules are applied to produce the posterior distribution for the parameter and heading pair $P_\theta(\pi|y)$ where the number of trials and "collisions" are incremented.


Hence the probability of collision $P_\theta(y|\pi)$ can be calculated by the expected value of the proportion $\pi$. Using the beta-binomial distribution the expected value of $\pi$ can be estimated using 

$$
\hat{\pi}_\theta = \frac{\alpha}{\alpha + \beta}
$$
where the update rules applied are
$$
\alpha_\theta = \alpha + y
$$
$$
\beta_\theta = \beta + n - y
$$
$$
p_\theta(y|\pi) \propto \pi^{y+\alpha - 1}(1-\pi)^{\beta + n-y - 1}
$$

Where $y \in Y$ is the expected number of collisions for the heading $\theta$.


The method _updateCollision_ is responsible for updating the parameter vectors for $\alpha_\theta$, $\beta_\theta$ and the collision count $y_\theta$ and $n_\theta$, while the method _selectLeastLikelihood_ is responsible for selecting the heading $\theta$ with the least likelihood of collision for the current sliding window (during selection the likelihood is calculated as $y = 1$ and $n = N$ in $P_\theta(y|\pi) \sim Binom(y, n, \pi) )$. 


Using the bayesian update rules for the beta-binomial, a kind of memory for the sliding window is created which allows the angle $\theta$ to be used for the heading to keep track of successful or unsuccessful selections for $n = N$. The method _changeDirection_ makes use of the _selectLeastLikelihood_ in order to explore and gradually learn the headings which are currently most likely to result in __successful__ movement (any $RollCommand(\theta, velocity)$ that results in travelling at least the minimum distance threshold). 

Note that this is a short term memory depending on the size of $N$. Additionally the _selectLeastLikelihood_ method takes a $min$ and $max$ parameter to filter the allowed selection of possible $\theta$.

Further experimentation will also make use of some form of localisation such as a graph which collects coordinates recorded from the odometer, and maps those headings most likely to result in a collision to the associated coordinates. However this type of experimentation has not yet been implemented, as localisation has not yet been attempted (but will be at a later time).



